# Решение тестового задания авито

## Описание задачи
На Авито пользователи часто вводят тексты в поиске, описаниях или заголовках с опечатками, пропущенными пробелами или слитным написанием слов (например: книгавхорошемсостоянии).Такие тексты усложняют понимание, снижают качество поиска и мешают автоматическому анализу (например, извлечению сущностей). Да, можно просто использовать дорогую LLM модель, но как будто такая задача может решаться гораздо более дешевым и быстрым способом. Мы ищем именно такое решение - точное, быстрое и легковесное.

Ваша задача — разработать модель или алгоритм, который принимает на вход текст без пробелов и возвращает восстановленный текст с правильными пробелами и позициями, где они были пропущены. Обратите внимание, ваше решение ОБЯЗАНО запускаться без проблем проверяющим. Невозможность запуска и подтверждения результатов полученной метрики - обнуление достигнутой метрики.



## Определение метрики

Качество оценивается по F1-score для позиций пропущенных пробелов.

Для каждой строки мы сравниваем множество позиций, где вы поставили пробелы, с истинным множеством.

Формула:

$F1 = 2 * (precision * recall) / (precision + recall)$

                  
Где:

precision = |предсказанные ∩ истинные| / |предсказанные|

recall = |предсказанные ∩ истинные| / |истинные|

Окончательная метрика — средний F1 по всем текстам.

На Stepik вы будете видеть F1 в процентах (от 0 до 100).

[пример файла submission](https://ucarecdn.com/81985c03-e860-4ca2-b6b2-876e9356aa9b/)

## Установка зависимостей

In [1]:
!pip -q install -U pandas scikit-learn datasets
from typing import List, Tuple
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


## считывание данных

In [2]:
import pandas as pd

path = "dataset_1937770_3.txt"

df = pd.DataFrame(
    [line.rstrip("\n").split(",", 1) 
     for line in open(path, encoding="utf-8") 
     if line.strip() and not line.lower().startswith("id,")],
    columns=["id", "text_no_spaces"]
)
df["id"] = df["id"].astype(int)

print(df.head())
print(df.shape)

   id                 text_no_spaces
0   0                куплюайфон14про
1   1             ищудомвПодмосковье
2   2  сдаюквартирусмебельюитехникой
3   3     новыйдивандоставканедорого
4   4                 отдамдаромкошк
(1005, 2)


## Основная концепция

**Машинное обучение на признаках границ между символами** - вместо сложных языковых моделей используется классический подход с извлечением признаков и логистической регрессией.

## Ключевые компоненты решения:

### 1. **Представление задачи как бинарной классификации**
- Каждая позиция между соседними символами рассматривается как потенциальное место для пробела
- Модель предсказывает вероятность того, что в данной позиции должен стоять пробел

### 2. **Богатый набор признаков для каждой границы**
Для позиции между символами `i` и `i+1` извлекаются:
- **Базовые**: биграмма символов, типы символов (русские/латинские/цифры)
- **Контекстные**: окно из 5 символов вокруг позиции
- **Лингвистические**: смены алфавитов, регистра, гласные/согласные переходы
- **Структурные**: позиция в строке, длины слов, пунктуация
- **Паттерны**: CamelCase, аббревиатуры, числовые последовательности

### 3. **Обучение на синтетических данных**
- Создается корпус типичных объявлений Авито (техника, недвижимость, услуги)
- Из текстов с пробелами генерируются пары: текст без пробелов → позиции пробелов

### 4. **Пост-обработка правилами пунктуации**
- Корректировка предсказаний согласно правилам русского языка
- Обработка знаков препинания, тире, кавычек
- Удаление ложных пробелов в числах

## Преимущества подхода:

- **Быстрота**: логистическая регрессия работает значительно быстрее больших языковых моделей
- **Интерпретируемость**: можно понять, какие признаки важны для модели
- **Контролируемость**: легко добавлять правила и корректировки
- **Легковесность**: не требует больших вычислительных ресурсов

Это элегантное решение, которое использует классические методы ML с тщательно продуманными признаками для решения специфической задачи восстановления пробелов в объявлениях.

In [3]:
train_lines = [
    # Бытовая техника
    "куплю айфон 14 про",
    "куплю айфон 15 про макс",
    "куплю samsung galaxy s23 ultra",
    "новая микроволновка Samsung",
    "куплю холодильник LG",
    "срочно куплю стиральную машину Bosch",
    "продам телевизор Sony 42 дюйма",
    "куплю ноутбук HP",
    "новый ноутбук Asus доставка сегодня",
    "срочно продам ноутбук Lenovo",
    "куплю пылесос Dyson",
    "куплю робот пылесос Xiaomi",
    "куплю монитор Philips 27 дюймов",
    "продам видеокарту RTX 3060 ti",
    "куплю процессор Intel i7",
    "куплю оперативную память 16 ГБ DDR4",
    "куплю телефон Huawei недорого",
    "новый смартфон Realme 11 pro",
    "куплю планшет iPad бу",
    "срочно продам наушники JBL",
    "куплю колонки Edifier 5.1",
    "куплю фотоаппарат Canon EOS",
    "куплю принтер HP LaserJet",

    # Недвижимость
    "ищу дом в Подмосковье",
    "сдаю квартиру с мебелью и техникой",
    "сдам комнату студентке",
    "ищу квартиру у метро",
    "сдам студию без посредников",
    "сдаю офис в аренду",
    "сдам парковку в центре города",
    "сдам гараж на длительный срок",
    "ищу квартиру посуточно Москва",
    "сдаю дом с баней",
    "сдам квартиру молодоженам",
    "сдам студию в центре города",
    "срочно ищу комнату для девушки",
    "ищу квартиру в москве недорого",

    # Услуги
    "ищу грузчиков для переезда",
    "нужен мастер по ремонту стиралок",
    "ремонт квартир под ключ",
    "ищу репетитора по биологии",
    "ищу репетитора по математике",
    "ищу репетитора по английскому языку",
    "ищу программиста python",
    "ищу работу курьером",
    "срочно ищу работу поваром",
    "нужен электрик срочно",
    "нужен сантехник для ремонта",
    "ищу дизайнера интерьера",
    "ищу фотографа на мероприятие",
    "ищу няню для ребенка",
    "ищу уборщицу на выходные",
    "ищу парикмахера на дом",
    "ищу адвоката консультация",

    # Мебель / одежда
    "новый диван доставка недорого",
    "куплю шкаф купе бу",
    "продам кресло офисное",
    "сдам комод белый",
    "куплю кровать двухспальную",
    "новый стол и стулья",
    "куплю детскую кроватку бу",
    "куплю ковер шерстяной",
    "новая куртка зимняя доставка",
    "продам кожаную куртку бу",
    "куплю кроссовки Adidas оригинал",
    "куплю платье вечернее",
    "куплю джинсы Levi’s",

    # Транспорт
    "куплю велосипед Merida",
    "срочно продам велосипед Stels",
    "куплю машину ВАЗ бу на ходу",
    "куплю зимние шины",
    "куплю самокат детский",
    "ищу такси до аэропорта",
    "куплю лодку резиновую",
    "куплю скейтборд новый",
    "куплю мотоцикл бу",
    "куплю запчасти на ВАЗ 2107",

    # Разное
    "отдам даром кошку",
    "отдам даром старый шкаф",
    "суши доставка ночью",
    "ремонт iPhone быстро",
    "Москва Сбербанк банкомат рядом",
    "холодильник сломался помогите срочно",
    "хочу заказать такси в аэропорт",
    "хочу купить мороженое",
    "хочу заказать суши",
    "нужна замена стекла на смартфоне",
    "продаю детскую куртку осень весна",
    "ищу квартиру рядом с лесопарком",
    "куплю робот пылесос недорого",
    "сдаю студию возле набережной",
    "ремонт телевизоров на дому быстро",
    "новая книга фантастика в мягкой обложке",
    "срочно требуется няня с опытом",
    "куплю кресло мешок большое",
    "ищу учителя по физике для егэ",
    "сдам комнату недалеко от университета",
    "куплю холодильник beko маленький",
    "нужна доставка пианино на выходных",
    "сдам гараж с электричеством",
    "куплю ноутбук для учебы не игровой",
    "ищу собаку корги девочку",
    "новая плита gorenje гарантия год",
    "сдаю парковочное место в центре",
    "ремонт стиральных машин инверторных",
    "куплю электросамокат с пробегом до тысячи",
    "ищу репетитора по химии для восьмого класса",
    "продам велотренажер складной",
    "новый тахометр для мотоцикла",
    "куплю микроволновую печь beko",
    "ищу мастера по укладке ламината",
    "сдам кладовую в паркинге",
    "куплю наушники panasonic с микрофоном",
    "ремонт ноутбуков msi rog",
    "нужен программист на pet проект",
    "сдаю дачу у озера без посредников",
    "куплю чайник электрический polaris",
    "ищу парикмахера мужские стрижки",
    "продам объектив 50 мм canon",
    "новая мультиварка moulinex с чешской розеткой",
    "куплю xbox series x контроллер",
    "ищу фотографа на выпускной",
    "сдам помещение под мастерскую",
    "куплю принтер лазерный чб",
    "нужен электрик установить автоматы",
    "ремонт кофемашин delonghi быстро",
    "куплю гитару классическую alhambra",
    "ищу учителя по испанскому для начинающих",
    "сдаю комнату возле парка победы",
    "куплю велосипед trek cross",
    "новый монитор aoc 24 дюйма",
    "куплю роутер keenetic viva",
    "ищу щенка самоеда мальчика",
    "сдам офис на первом этаже",
    "куплю видеокарту rtx 3060 ti",
    "продам руль для симрейсинга",
    "нужен курьер с авто частичная занятость",
    "куплю матрас 160 на 200 средней жесткости",
    "ремонт духовых шкафов на дому",
    "ищу логопеда занятия онлайн",
    "сдаю студию на две недели",
    "куплю стабилизатор напряжения на дом",
    "новая сковорода гриль чугун",
    "продам гироскутер почти новый",
    "куплю процессор ryzen пятого поколения",
    "ищу уборщицу два раза в неделю",
    "сдам комнату девушке без вредных привычек",
    "куплю шкаф купе двухдверный",  
    "ремонт айпадов и макбуков",
    "ищу врача невролога консультация",
    "новая кофеварка saeco с капучинатором",
    "куплю электронное пианино yamaha p45",
    "сдам место в хостеле центр города",
    "куплю системный блок для офиса",
    "ищу дом на берегу реки",
    "продам кухню б у недорого",
    "нужна замена ремня грм на шкода",
    "куплю телевизор tcl qled сорок шесть",
    "ремонт проекторов epson sony",
    "ищу юриста для договора аренды",
    "сдам гараж сухой и теплый",
    "куплю сварочный инвертор fubag",
    "новая вытяжка kuchina гарантия",
    "продам стеллаж металлический высокий",
    "ищу учителя по программированию на python",
    "куплю смартфон realme десятый pro",
    "сдаю рабочее место в коворкинге",
    "ремонт холодильников hitachi на выезде",
    "куплю комод белый икеа малм",
    "ищу авто для свадьбы кабриолет",
    "сдам квартиру студию возле технопарка",
    "куплю акустическую систему jbl partybox",
    "продам моноколесо без царапин",
    "ищу мастера по плитке ванная",
    "нужна доставка мебели сегодня вечером",
    "куплю игровую консоль sega mega drive",
    "сдаю офис опенспейс пятьдесят метров",
    "ремонт смартфонов xiaomi mi",
    "куплю объектив sigma арт тридцать пять",
    "ищу репетитора по алгебре девятый класс",
    "новый чайник tefal с терморегулятором",
    "куплю кресло офисное эргономичное",
    "сдам место на стоянке под мотоцикл",
    "куплю стиральную машину узкую candy",
    "ремонт бойлеров ariston",
    "ищу швью на подработку",
    "продам лыжи rossignol рост сто семьдесят",
    "куплю планшет lenovo для рисования",
    "ищу монтажника кондиционеров",
    "сдаю павильон на рынке",
    "куплю лампу настольную с димером",
    "ремонт электрогитар с настройкой мензуры",
    "ищу тренера по плаванию взрослому",
    "новая духовая печь beko черная",
    "куплю фотоаппарат sony a6400",
    "сдам парковочное место крытое",
    "куплю шуруповерт makita двадцать вольт",
    "ищу няню для грудничка",
    "ремонт ноутбуков acer swift",
    "куплю чайный сервиз двенадцать персон",
    "сдаю комнату рядом с метро юго западная",
    "куплю кресло коляску для пожилого",
    "ищу мастера по ремонту обуви",
    "новый холодильник indesit с ноуфрост",
    "куплю монитор philips двадцать семь ips",
    "сдам кладовку в новом доме",
    "куплю кухонный комбайн bosch компакт",
    "ищу врача терапевта дом на дом",
    "ремонт смартфонов huawei honor",
    "куплю камеру видеонаблюдения ezviz",
    "сдаю комнату студенту айти",
    "куплю телефон nokia кнопочный",
    "ищу собаку шельти в семью",
    "новая стиральная машина hisense инвертор",
    "куплю пылесос redmond циклонный",
    "сдам однушку на длительный срок",
    "куплю рюкзак туристический сорок литров",
    "ищу веб разработчика на проект",
    "ремонт посудомоек bosch siemens",
    "куплю детскую кроватку с маятником",
    "сдаю место в офисе для фрилансера",
    "куплю видеорегистратор с двумя камерами",
    "ищу учителя по китайскому онлайн",
    "новая люстра на пять ламп",
    "куплю наушники беспроводные oneodio",
    "сдам студию у парка три недели",
    "куплю модем lte с антеннами",
    "ремонт кондиционеров mitsubishi heavy",
    "ищу фотографа для предметной съемки",
    "куплю холодильник sharp без следов",
    "сдаю апартаменты в новом жк",
    "куплю палатки три местные trimm",
    "ищу сантехника заменить смеситель",
    "новый пылесос kitfort вертикальный",
    "куплю кресло компьютерное с сеткой",
    "сдам дом в коттеджном поселке",
    "куплю 3d принтер ender три",
    "ремонт геймпадов dualsense",
    "ищу электрика срочно после залива",
    "куплю коврик для йоги нескользящий",
    "сдаю теплое машиноместо в тц",
    "куплю кофемолку жерновую timemore",
    "ищу тренера по теннису",
    "новая варочная панель gas on glass",
    "куплю роутер zyxel keenetic giga",
    "сдам комнату без залога собственник",
    "куплю фотопринтер canon selphy",
    "ищу переводчика с французского",
    "ремонт духовых труб в духовке",
    "куплю усилитель звука marantz",
    "сдаю квартиру возле кампуса маи",
    "куплю колонки edifier r1280",
    "ищу мастера по натяжным потолкам",
    "новая мышь logitech mx performance",
    "куплю клавиатуру механическую hot swap",
    "сдам лофт под творческую студию",
    "куплю спиннинг шимано ультралайт",
    "ищу пекаря в ночную смену",
    "ремонт электроинструмента в день обращения",
    "куплю велосипед детский на семь лет",
    "сдаю место под рекламу фасад",
    "куплю часы casio g shock",
    "ищу врача стоматолога терапевта",
    "новый комод икеа хемнэс",
    "куплю роутер с вайфай шесть",
    "сдам офис кабинет сорок метров",
    "куплю микрофон fifine конденсаторный",
    "ищу моделлера трехд для игры",
    "куплю осушитель воздуха ballu",
    "сдаю киоск на остановке",
    "куплю увлажнитель воздуха boneco",
    "ремонт телефонов tecno infinix",
    "ищу гитарного мастера сетап инструмента",
    "куплю набор кастрюль сфера",
    "новая хлебопечка panasonic с дозатором",
    "куплю матрас 160 на 200 средней жесткости",
    "ремонт духовых шкафов на дому",
    "ищу логопеда занятия онлайн",
    "сдаю студию на две недели",
    "куплю стабилизатор напряжения на дом",
    "новая сковорода гриль чугун",
    "продам гироскутер почти новый",
    "куплю процессор ryzen пятого поколения",
    "ищу уборщицу два раза в неделю",
    "сдам комнату девушке без вредных привычек",
    "куплю шкаф купе двухдверный",
    "ремонт айпадов и макбуков",
    "ищу врача невролога консультация",
    "новая кофеварка saeco с капучинатором",
    "куплю электронное пианино yamaha p45",
    "сдам место в хостеле центр города",
    "куплю системный блок для офиса",
    "ищу дом на берегу реки",
    "продам кухню б у недорого",
    "нужна замена ремня грм на шкода",
    "куплю телевизор tcl qled сорок шесть",
    "ремонт проекторов epson sony",
    "ищу юриста для договора аренды",
    "сдам гараж сухой и теплый",
    "куплю сварочный инвертор fubag",
    "новая вытяжка kuchina гарантия",
    "продам стеллаж металлический высокий",
    "ищу учителя по программированию на python",
    "куплю смартфон realme десятый pro",
    "сдаю рабочее место в коворкинге",
    "ремонт холодильников hitachi на выезде",
    "куплю комод белый икеа малм",
    "ищу авто для свадьбы кабриолет",
    "сдам квартиру студию возле технопарка",
    "куплю акустическую систему jbl partybox",
    "продам моноколесо без царапин",
    "ищу мастера по плитке ванная",
    "нужна доставка мебели сегодня вечером",
    "куплю игровую консоль sega mega drive",
    "сдаю офис опенспейс пятьдесят метров",
    "ремонт смартфонов xiaomi mi",
    "куплю объектив sigma арт тридцать пять",
    "ищу репетитора по алгебре девятый класс",
    "новый чайник tefal с терморегулятором",
    "куплю кресло офисное эргономичное",
    "сдам место на стоянке под мотоцикл",
    "куплю стиральную машину узкую candy",
    "ремонт бойлеров ariston",
    "ищу швью на подработку",
    "продам лыжи rossignol рост сто семьдесят",
    "куплю планшет lenovo для рисования",
    "ищу монтажника кондиционеров",
    "сдаю павильон на рынке",
    "куплю лампу настольную с димером",
    "ремонт электрогитар с настройкой мензуры",
    "ищу тренера по плаванию взрослому",
    "новая духовая печь beko черная",
    "куплю фотоаппарат sony a6400",
    "сдам парковочное место крытое",
    "куплю шуруповерт makita двадцать вольт",
    "ищу няню для грудничка",
    "ремонт ноутбуков acer swift",
    "куплю чайный сервиз двенадцать персон",
    "сдаю комнату рядом с метро юго западная",
    "куплю кресло коляску для пожилого",
    "ищу мастера по ремонту обуви",
    "новый холодильник indesit с ноуфрост",
    "куплю монитор philips двадцать семь ips",
    "сдам кладовку в новом доме",
    "куплю кухонный комбайн bosch компакт",
    "ищу врача терапевта дом на дом",
    "ремонт смартфонов huawei honor",
    "куплю камеру видеонаблюдения ezviz",
    "сдаю комнату студенту айти",
    "куплю телефон nokia кнопочный",
    "ищу собаку шельти в семью",
    "новая стиральная машина hisense инвертор",
    "куплю пылесос redmond циклонный",
    "сдам однушку на длительный срок",
    "куплю рюкзак туристический сорок литров",
    "ищу веб разработчика на проект",
    "ремонт посудомоек bosch siemens",
    "куплю детскую кроватку с маятником",
    "сдаю место в офисе для фрилансера",
    "куплю видеорегистратор с двумя камерами",
    "ищу учителя по китайскому онлайн",
    "новая люстра на пять ламп",
    "куплю наушники беспроводные oneodio",
    "сдам студию у парка три недели",
    "куплю модем lte с антеннами",
    "ремонт кондиционеров mitsubishi heavy",
    "ищу фотографа для предметной съемки",
    "куплю холодильник sharp без следов",
    "сдаю апартаменты в новом жк",
    "куплю палатки три местные trimm",
    "ищу сантехника заменить смеситель",
    "новый пылесос kitfort вертикальный",
    "куплю кресло компьютерное с сеткой",
    "сдам дом в коттеджном поселке",
    "куплю 3d принтер ender три",
    "ремонт геймпадов dualsense",
    "ищу электрика срочно после залива",
    "куплю коврик для йоги нескользящий",
    "сдаю теплое машиноместо в тц",
    "куплю кофемолку жерновую timemore",
    "ищу тренера по теннису",
    "новая варочная панель gas on glass",
    "куплю роутер zyxel keenetic giga",
    "сдам комнату без залога собственник",
    "куплю фотопринтер canon selphy",
    "ищу переводчика с французского",
    "ремонт духовых труб в духовке",
    "куплю усилитель звука marantz",
    "сдаю квартиру возле кампуса маи",
    "куплю колонки edifier r1280",
    "ищу мастера по натяжным потолкам",
    "новая мышь logitech mx performance",
    "куплю клавиатуру механическую hot swap",
    "сдам лофт под творческую студию",
    "куплю спиннинг шимано ультралайт",
    "ищу пекаря в ночную смену",
    "ремонт электроинструмента в день обращения",
    "куплю велосипед детский на семь лет",
    "сдаю место под рекламу фасад",
    "куплю часы casio g shock",
    "ищу врача стоматолога терапевта",
    "новый комод икеа хемнэс",
    "куплю роутер с вайфай шесть",
    "сдам офис кабинет сорок метров",
    "куплю микрофон fifine конденсаторный",
    "ищу моделлера трехд для игры",
    "куплю осушитель воздуха ballu",
    "сдаю киоск на остановке",
    "куплю увлажнитель воздуха boneco",
    "ремонт телефонов tecno infinix",
    "ищу гитарного мастера сетап инструмента",
    "куплю набор кастрюль сфера",
    "новая хлебопечка panasonic с дозатором",
    "куплю матрас 160 на 200 средней жесткости",
    "ремонт духовых шкафов на дому",
    "ищу логопеда занятия онлайн",
    "сдаю студию на две недели",
    "куплю стабилизатор напряжения на дом",
    "новая сковорода гриль чугун",
    "продам гироскутер почти новый",
    "куплю процессор ryzen пятого поколения",
    "ищу уборщицу два раза в неделю",
    "сдам комнату девушке без вредных привычек",
    "куплю шкаф купе двухдверный",
    "ремонт айпадов и макбуков",
    "ищу врача невролога консультация",
    "новая кофеварка saeco с капучинатором",
    "куплю электронное пианино yamaha p45",
    "сдам место в хостеле центр города",
    "куплю системный блок для офиса",
    "ищу дом на берегу реки",
    "продам кухню б у недорого",
    "нужна замена ремня грм на шкода",
    "куплю телевизор tcl qled сорок шесть",
    "ремонт проекторов epson sony",
    "ищу юриста для договора аренды",
    "сдам гараж сухой и теплый",
    "куплю сварочный инвертор fubag",
    "новая вытяжка kuchina гарантия",
    "продам стеллаж металлический высокий",
    "ищу учителя по программированию на python",
    "куплю смартфон realme десятый pro",
    "сдаю рабочее место в коворкинге",
    "ремонт холодильников hitachi на выезде",
    "куплю комод белый икеа малм",
    "ищу авто для свадьбы кабриолет",
    "сдам квартиру студию возле технопарка",
    "куплю акустическую систему jbl partybox",  
    "продам моноколесо без царапин",
    "ищу мастера по плитке ванная",
    "нужна доставка мебели сегодня вечером",
    "куплю игровую консоль sega mega drive",
    "сдаю офис опенспейс пятьдесят метров",
    "ремонт смартфонов xiaomi mi",
    "куплю объектив sigma арт тридцать пять",
    "ищу репетитора по алгебре девятый класс",
    "новый чайник tefal с терморегулятором",
    "куплю кресло офисное эргономичное",
    "сдам место на стоянке под мотоцикл",
    "куплю стиральную машину узкую candy",
    "ремонт бойлеров ariston",
    "ищу швью на подработку",
    "продам лыжи rossignol рост сто семьдесят",
    "куплю планшет lenovo для рисования",
    "ищу монтажника кондиционеров",
    "сдаю павильон на рынке",
    "куплю лампу настольную с димером",
    "ремонт электрогитар с настройкой мензуры",
    "ищу тренера по плаванию взрослому",
    "новая духовая печь beko черная",
    "куплю фотоаппарат sony a6400",
    "сдам парковочное место крытое",
    "куплю шуруповерт makita двадцать вольт",
    "ищу няню для грудничка",
    "ремонт ноутбуков acer swift",
    "куплю чайный сервиз двенадцать персон",
    "сдаю комнату рядом с метро юго западная",
    "куплю кресло коляску для пожилого",
    "ищу мастера по ремонту обуви",
    "новый холодильник indesit с ноуфрост",
    "куплю монитор philips двадцать семь ips",
    "сдам кладовку в новом доме",
    "куплю кухонный комбайн bosch компакт",
    "ищу врача терапевта дом на дом",
    "ремонт смартфонов huawei honor",
    "куплю камеру видеонаблюдения ezviz",
    "сдаю комнату студенту айти",
    "куплю телефон nokia кнопочный",
    "ищу собаку шельти в семью",
    "новая стиральная машина hisense инвертор",
    "куплю пылесос redmond циклонный",
    "сдам однушку на длительный срок",
    "куплю рюкзак туристический сорок литров",
    "ищу веб разработчика на проект",
    "ремонт посудомоек bosch siemens",
    "куплю детскую кроватку с маятником",
    "сдаю место в офисе для фрилансера",
    "куплю видеорегистратор с двумя камерами",
    "ищу учителя по китайскому онлайн",
    "новая люстра на пять ламп",
    "куплю наушники беспроводные oneodio",
    "сдам студию у парка три недели",
    "куплю модем lte с антеннами",
    "ремонт кондиционеров mitsubishi heavy",
    "ищу фотографа для предметной съемки",
    "куплю холодильник sharp без следов",
    "сдаю апартаменты в новом жк",
    "куплю палатки три местные trimm",
    "ищу сантехника заменить смеситель",
    "новый пылесос kitfort вертикальный",
    "куплю кресло компьютерное с сеткой",
    "сдам дом в коттеджном поселке",
    "куплю 3d принтер ender три",
    "ремонт геймпадов dualsense",
    "ищу электрика срочно после залива",
    "куплю коврик для йоги нескользящий",
    "сдаю теплое машиноместо в тц",
    "куплю кофемолку жерновую timemore",
    "ищу тренера по теннису",
    "новая варочная панель gas on glass",
    "куплю роутер zyxel keenetic giga",
    "сдам комнату без залога собственник",
    "куплю фотопринтер canon selphy",
    "ищу переводчика с французского",
    "ремонт духовых труб в духовке",
    "куплю усилитель звука marantz",
    "сдаю квартиру возле кампуса маи",
    "куплю колонки edifier r1280",      
    "ищу мастера по натяжным потолкам",
    "новая мышь logitech mx performance",
    "куплю клавиатуру механическую hot swap",
    "сдам лофт под творческую студию",
    "куплю спиннинг шимано ультралайт",
    "ищу пекаря в ночную смену",
    "ремонт электроинструмента в день обращения",
    "куплю велосипед детский на семь лет",
    "сдаю место под рекламу фасад",
    "куплю часы casio g shock",
    "ищу врача стоматолога терапевта",
    "новый комод икеа хемнэс",
    "куплю роутер с вайфай шесть",
    "сдам офис кабинет сорок метров",
    "куплю микрофон fifine конденсаторный",
    "ищу моделлера трехд для игры",
    "куплю осушитель воздуха ballu",
    "сдаю киоск на остановке",
    "куплю увлажнитель воздуха boneco",
    "ремонт телефонов tecno infinix",
    "ищу гитарного мастера сетап инструмента",
    "куплю набор кастрюль сфера",
    "новая хлебопечка panasonic с дозатором"

]

### Формирование целевых метрик для модели

In [4]:
def remove_spaces_and_targets(s: str) -> Tuple[str, List[int]]:
    chars, y = [], []
    for ch in s:
        if ch == " ":
            if y: y[-1] = 1   # если был пробел → метка "1" у предыдущего символа
        else:
            chars.append(ch)
            y.append(0)
    if y: y[-1] = 0  # у последнего символа пробела нет
    return "".join(chars), y

### Вспомогательные переменные и функции

In [5]:
RUS_VOWELS  = set("аеёиоуыэюяАЕЁИОУЫЭЮЯ")
RUS_LETTERS = set("абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ")
LAT_LETTERS = set("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ")
DIGITS      = set("0123456789")

In [6]:
# Проверка типа символа
def ctype(ch: str) -> str:
    if ch in RUS_LETTERS: return "RU"
    if ch in LAT_LETTERS: return "EN"
    if ch in DIGITS:      return "DG"
    return "OT"

In [7]:
# Проверка гласной буквы
def is_vowel(ch: str) -> int:
    return int(ch in RUS_VOWELS)

### !!!Главная идея: каждую границу мы описываем признаками «что слева и справа», «какой контекст рядом», «какие переходы по типам букв». Это даёт модели материал для предсказания пробела.

#### Пример признаков для границы между i и i+1 символами строки s:
```json
{
 'bigram': 'н|1',
 'lt': 'RU', 'rt': 'DG',
 'type_change': 1,
 'case_change': 0,
 'digit_to_alpha': 0,
 'alpha_to_digit': 1,
 'vowel_to_cons': 0,
 'cons_to_vowel': 0,
 'c-2': 'о', 'c-2_t': 'RU', 'c-2_u': 0, 'c-2_v': 1,
 'c-1': 'н', 'c-1_t': 'RU', 'c-1_u': 0, 'c-1_v': 0,
 'c0': 'н', 'c0_t': 'RU', 'c0_u': 0, 'c0_v': 0,
 'c1': '1', 'c1_t': 'DG', 'c1_u': 0, 'c1_v': 0,
 'c2': '4', 'c2_t': 'DG', 'c2_u': 0, 'c2_v': 0
}
```

In [8]:
import re
# Признаки для границы между символами s[i] и s[i+1]
def feats_for_boundary(s: str, i: int, win: int = 2) -> dict:
    n = len(s)
    left, right = s[i], s[i+1]
    f = {
        # --- базовые ---
        "bigram": left + "|" + right,
        "lt": ctype(left),
        "rt": ctype(right),
        "type_change": int(ctype(left) != ctype(right)),
        "case_change": int(left.isupper() != right.isupper()),
        "digit_to_alpha": int(left in DIGITS and right not in DIGITS),
        "alpha_to_digit": int(left not in DIGITS and right in DIGITS),
        "vowel_to_cons": int(is_vowel(left) and (right in RUS_LETTERS and not is_vowel(right))),
        "cons_to_vowel": int((left in RUS_LETTERS and not is_vowel(left)) and is_vowel(right)),

        # --- расширенные ---
        "trigram_l": (s[i-1] if i > 0 else "<BOS>") + left + "|" + right,
        "trigram_r": left + "|" + right + (s[i+2] if i+2 < n else "<EOS>"),
        "lt_rt_types": ctype(left) + "_" + ctype(right),
        "both_en": int(left in LAT_LETTERS and right in LAT_LETTERS),
        "both_ru": int(left in RUS_LETTERS and right in RUS_LETTERS),
        "vc_pattern": str(is_vowel(left)) + str(is_vowel(right)),
        "left_punct": int(not left.isalnum()),
        "right_punct": int(not right.isalnum()),
        "digit_seq": int(left in DIGITS and right in DIGITS),
        "camel_case": int(left.islower() and right.isupper()),
        "abbr": int(left.isupper() and right.isupper()),
        "len_left_word": sum(1 for j in range(i, -1, -1) if s[j].isalpha()),
        "len_right_word": sum(1 for j in range(i+1, n) if s[j].isalpha()) if right.isalpha() else 0,

        # --- новые ---
        # Позиция в строке
        "pos_norm": i / n,
        "is_near_start": int(i < 3),
        "is_near_end": int(i > n - 4),

        # Юникод-классы
        "left_isalpha": int(left.isalpha()),
        "right_isalpha": int(right.isalpha()),
        "left_isdigit": int(left.isdigit()),
        "right_isdigit": int(right.isdigit()),
        "left_isspace": int(left.isspace()),
        "right_isspace": int(right.isspace()),
        "left_ispunct": int(re.match(r"\W", left) is not None),
        "right_ispunct": int(re.match(r"\W", right) is not None),

        # Паттерны смены алфавитов
        "ru_to_en": int(left in RUS_LETTERS and right in LAT_LETTERS),
        "en_to_ru": int(left in LAT_LETTERS and right in RUS_LETTERS),
        "digit_to_ru": int(left in DIGITS and right in RUS_LETTERS),
        "ru_to_digit": int(left in RUS_LETTERS and right in DIGITS),

        # Регистр
        "upper_to_lower": int(left.isupper() and right.islower()),
        "lower_to_upper": int(left.islower() and right.isupper()),

        # Кавычки и скобки
        "left_quote": int(left in "\"'«»"),
        "right_quote": int(right in "\"'«»"),
        "left_bracket": int(left in "(["),
        "right_bracket": int(right in ")]"),
    }

    # --- контекстное окно ---
    for off in range(-win, win+1):
        j = i+off
        key = f"c{off}"
        if 0 <= j < n:
            ch = s[j]
            f[key]     = ch
            f[key+"_t"]= ctype(ch)
            f[key+"_u"]= int(ch.isupper())
            f[key+"_v"]= is_vowel(ch)
        else:
            f[key]     = "<PAD>"
            f[key+"_t"]= "PAD"
            f[key+"_u"]= 0
            f[key+"_v"]= 0
    return f

In [9]:
def make_train(lines: List[str], max_len=2000):
    X, y = [], []
    for s in lines:
        ns, tgt = remove_spaces_and_targets(s) # ns - строка без пробелов, tgt - метки
        for i in range(len(ns)-1):
            X.append(feats_for_boundary(ns, i))
            y.append(tgt[i])
    return X, y # X - признаки, y - метки

In [10]:
def build_model() -> Pipeline:
    clf = LogisticRegression(
        solver="liblinear", penalty="l2", C=1.0,
        max_iter=200, class_weight="balanced", random_state=42
    )
    return Pipeline([("vec", DictVectorizer()), ("lr", clf)]) # модель с преобразованием признаков

In [11]:
def restore_spaces(model: Pipeline, s: str, thr: float = 0.45) -> str:
    if not s or len(s) == 1: return s
    feats = [feats_for_boundary(s, i) for i in range(len(s)-1)]
    probs = model.predict_proba(feats)[:, 1]
    out = [s[0]]
    for i, p in enumerate(probs):
        if p >= thr: out.append(" ")
        out.append(s[i+1])
    return "".join(out)

In [12]:
# Обучаем модель на мини-корпусе
X_train, y_train = make_train(train_lines)
model = build_model().fit(X_train, y_train)

print(f"Обучено на примерах: {len(y_train)}")

Обучено на примерах: 14233


In [13]:
import re
import pandas as pd
from typing import List, Tuple, Set


def strip_spaces(s: str) -> str:
    """Убираем все пробелы (на случай, если в исходнике попались)."""
    return "".join(ch for ch in s if ch != " ")

def positions_from_model(model, s0: str, thr: float = 0.5) -> Set[int]:
    """
    Возвращает множество позиций (индексов), куда вставлять пробел, для строки s0 без пробелов.
    Позиция i означает пробел МЕЖДУ s0[i-1] и s0[i].
    """
    if len(s0) <= 1:
        return set()
    feats = [feats_for_boundary(s0, i) for i in range(len(s0)-1)]
    proba = model.predict_proba(feats)[:, 1]
    # если p>=thr, вставляем пробел ПОСЛЕ символа с индексом i => позиция i+1
    pos = {i+1 for i, p in enumerate(proba) if p >= thr}
    # не ставим пробел на позиции 0
    pos.discard(0)
    # приводим к отсортированному виду при выводе
    return set(sorted(pos))

In [14]:
PUNCT_NO_SPACE_BEFORE = set(",;:.?!)]»")
PUNCT_SPACE_AFTER     = set(",;:?!)»")
DASHES                = "—–-"

def postprocess_positions(s0: str, pos_set: Set[int]) -> Set[int]:
    """
    Корректируем предсказанные позиции простыми правилами пунктуации.
    s0 — строка БЕЗ пробелов.
    """
    n = len(s0)
    pos = set(pos_set)

    # 1) убрать пробел ПЕРЕД знаками, где он не нужен: " , . ; : ? ! ) »"
    for i in list(pos):
        if 0 <= i < n and s0[i] in PUNCT_NO_SPACE_BEFORE:
            pos.discard(i)

    # 2) добавить пробел ПОСЛЕ некоторых знаков, если далее буква/цифра/открывающая кавычка/скобка
    for i, ch in enumerate(s0):
        if ch in PUNCT_SPACE_AFTER and i+1 < n:
            if s0[i+1].isalnum() or s0[i+1] in "«(\"'[":
                pos.add(i+1)

    # 3) тире — пробел по обе стороны, если по краям буквы/цифры
    for i, ch in enumerate(s0):
        if ch in DASHES:
            if i > 0 and s0[i-1].isalnum():
                pos.add(i)
            if i+1 < n and s0[i+1].isalnum():
                pos.add(i+1)

    # 4) «5,3» внутри числа — НЕ добавлять пробел после запятой
    for i in list(pos):
        if 0 <= i-1 < n and 0 <= i < n:
            if s0[i-1].isdigit() and s0[i] == ',' and i+1 < n and s0[i+1].isdigit():
                pos.discard(i+1)

    pos.discard(0)
    return set(sorted(pos))

In [15]:
def restore_by_positions(s0: str, positions: Set[int]) -> str:
    """Из s0 и множества позиций собираем текст с пробелами (удобно для отладки)."""
    out = []
    for i, ch in enumerate(s0):
        if i in positions:
            out.append(" ")
        out.append(ch)
    return "".join(out)

def predict_positions_with_pp(model, s: str, thr: float = 0.55) -> List[int]:
    """
    Полная обёртка: убираем пробелы, предсказываем позиции, делаем пост-процессинг.
    Возвращаем ОТСОРТИРОВАННЫЙ список индексов.
    """
    s0 = strip_spaces(str(s))
    pos = positions_from_model(model, s0, thr=thr)
    pos = postprocess_positions(s0, pos)
    return sorted(pos)

In [16]:
import re
from typing import List, Iterable

# --- полезные константы ---

BRANDS = {
    # латиница (встречаются в объявлениях)
    "iphone","ipad","macbook","imac","airpods",
    "samsung","lg","sony","tcl","huawei","honor","oppo","xiaomi","redmi","realme",
    "bosch","indesit","atlant","dyson","jbl","tp-link","tplink","msi","asus","dell","lenovo","acer","hp",
    "gibson","fender","yamaha","merida","stels","giant","casio","lego","redmond",
    "playstation","ps","xbox","nintendo","rtx","gtx","intel","amd","nvidia",
    "philips","toshiba","sharp","tcl","olympus","canon","nikon","pixel","oneplus",
    # кириллица бренды/торговые названия (иногда пишут латиницей и кириллицей вперемешку)
    "икеа","икея","сбербанк","яндекс","авито"  # на всякий случай
}

# для быстрой проверки префиксных совпадений по брендам
BRANDS_SORTED = sorted(BRANDS, key=len, reverse=True)

# юнит-паттерны: пробел перед словом-единицей
UNITS = r"(дюйм(?:ов|а)?|см|мм|м|кг|г|мг|л|литр(?:ов|а)?|гб|тб|gb|tb)"
RE_NUM_UNITS = re.compile(rf"(\d+){UNITS}", flags=re.IGNORECASE)

# буквы+цифры в одной «склейке» → пробел между ними (для моделей)
RE_LET_DIG = re.compile(r"([A-Za-zА-Яа-яЁё]+)(\d+)")
RE_DIG_LET = re.compile(r"(\d+)([A-Za-zА-Яа-яЁё]+)")

# более специфичные игровые / «железные» шаблоны
# RTX4060Ti, GTX1080, iPhone15Pro, LG55, PS5, XboxSeriesS
RE_GPU = re.compile(r"(rtx|gtx)(\d{3,4}[a-z]{0,3})", flags=re.IGNORECASE)
RE_IPHONE = re.compile(r"(iphone)(\d{1,2}[a-z]*)", flags=re.IGNORECASE)
RE_LG_NUM = re.compile(r"(lg)(\d{2,3})", flags=re.IGNORECASE)
RE_PS = re.compile(r"(ps)(\d{1,2})", flags=re.IGNORECASE)
RE_XBOX_SERIES = re.compile(r"(xbox)(series)([a-z])?", flags=re.IGNORECASE)

# пунктуация — на этих символах пробел перед ними обычно не нужен
PUNCT = set(",.!?:;)]}»")


def _add(pos_set: set, i: int, n: int):
    """Добавить позицию i (между s[i-1] и s[i]) если в допустимых границах."""
    if 1 <= i <= n-1:
        pos_set.add(i)


def _regex_split_positions(s: str, pat: re.Pattern, group_left: int, group_right: int) -> Iterable[int]:
    """
    Возвращает позиции для пробела между концом group_left и началом group_right
    в матче паттерна pat.
    """
    for m in pat.finditer(s):
        i = m.start(group_right)  # позиция правой группы = индекс вставки пробела
        yield i


def _brand_boundaries(s: str) -> List[int]:
    """
    Найти места, где начинается бренд (латиница/кириллица) сразу после «русского» слова/цифр,
    и поставить пробел перед брендом.
    """
    s_low = s.lower()
    n = len(s)
    pos = set()
    for start in range(n):
        # экономим: проверяем только там, где смена типа (напр. кириллица -> латиница) или буква после буквы
        ch = s_low[start]
        if not ch.isalnum():
            continue
        # ищем максимальный бренд, который совпадает с префиксом s_low[start:]
        for b in BRANDS_SORTED:
            if s_low.startswith(b, start):
                # условие разумной границы: слева должен быть буква/цифра и это НЕ начало строки
                if start > 0 and s[start-1].isalnum():
                    _add(pos, start, n)
                # а ещё часто нужен пробел ПОСЛЕ бренда и ПЕРЕД цифрой/моделью: Samsung55 → Samsung 55
                end = start + len(b)
                if end < n and s[end].isdigit():
                    _add(pos, end, n)
                break  # нашёл самый длинный бренд — дальше не проверяем
    return sorted(pos)


def _letter_digit_boundaries(s: str) -> List[int]:
    """Пробелы между буквенной и цифровой частью и обратно (A54 → A 54, 55дюймов → 55 дюймов)."""
    pos = set()
    n = len(s)
    for m in RE_LET_DIG.finditer(s):
        _add(pos, m.start(2), n)  # перед цифрами
    for m in RE_DIG_LET.finditer(s):
        _add(pos, m.start(2), n)  # перед буквами
    # юниты: 55дюймов → 55 дюймов, 70литров → 70 литров, 128gb → 128 gb
    for m in RE_NUM_UNITS.finditer(s):
        _add(pos, m.start(2), n)
    return sorted(pos)


def _model_specific_boundaries(s: str) -> List[int]:
    """Спец-кейсы для RTX/GTX, iPhone, LG55, PS5, Xbox Series S."""
    pos = set()
    n = len(s)
    for i in _regex_split_positions(s, RE_GPU, 1, 2):
        _add(pos, i, n)
    for i in _regex_split_positions(s, RE_IPHONE, 1, 2):
        _add(pos, i, n)
    for i in _regex_split_positions(s, RE_LG_NUM, 1, 2):
        _add(pos, i, n)
    for i in _regex_split_positions(s, RE_PS, 1, 2):
        _add(pos, i, n)
    # xbox + series + S|X…
    for m in RE_XBOX_SERIES.finditer(s):
        _add(pos, m.start(2), n)  # ...xbox|series...
        if m.group(3):            # ...series|s
            _add(pos, m.start(3), n)
    return sorted(pos)


def _clean_punct_positions(s: str, positions: Iterable[int]) -> List[int]:
    """
    Убрать пробелы ПЕРЕД пунктуацией и ДВОЙНЫЕ пробелы (две соседние позиции).
    Позиция i вставляет пробел между s[i-1] и s[i].
    """
    n = len(s)
    pos = sorted(set(p for p in positions if 1 <= p <= n-1))

    cleaned = []
    last = -10
    for p in pos:
        # не ставим пробел перед пунктуацией
        if s[p] in PUNCT:
            continue
        # не делаем двойные пробелы вплотную
        if p == last + 1:
            # оставим тот, у которого «правый» символ не пунктуация (мы уже отфильтровали),
            # в споре — оставляем первый
            continue
        cleaned.append(p)
        last = p
    return cleaned


def apply_domain_rules(s_no_spaces: str, positions: List[int]) -> List[int]:
    """
    Доменные пост-правила: возвращает обновлённый список позиций.
    """
    n = len(s_no_spaces)
    base = set(positions)

    # 1) Буквы/цифры/юниты
    for i in _letter_digit_boundaries(s_no_spaces):
        base.add(i)

    # 2) Бренды (начало бренда / бренд -> цифра)
    for i in _brand_boundaries(s_no_spaces):
        base.add(i)

    # 3) Специализированные модели
    for i in _model_specific_boundaries(s_no_spaces):
        base.add(i)

    # 4) Чистка пунктуации и двойных пробелов
    out = _clean_punct_positions(s_no_spaces, base)

    return sorted(out)

In [17]:
def predict_positions_with_rules(model, s: str, thr: float) -> List[int]:
    # s здесь — ИСХОДНАЯ строка без пробелов (как у Stepik)
    # 1) базовый предикт (твой текущий)
    base_pos = predict_positions_with_pp(model, s, thr=thr)  # или restore → обратно в позиции

    # 2) доменные правила
    pos = apply_domain_rules(s, base_pos)
    return pos

In [18]:
# df — это исходный task_data: колонки ['id', 'text_no_spaces']
# ВАЖНО: колонка predicted_positions — ТИП ДАННЫХ СТРОКА, например "[1, 14]"

from typing import List

def to_str_list(indices: List[int]) -> str:
    return "[" + ", ".join(str(i) for i in indices) + "]"

def make_submission(df: pd.DataFrame, model, thr: float = 0.55) -> pd.DataFrame:
    preds_str = []
    for s in df["text_no_spaces"].astype(str):
        s0 = s  # уже без пробелов по условию датасета
        pos = predict_positions_with_rules(model, s0, thr=thr)
        preds_str.append("[" + ", ".join(map(str, pos)) + "]")
    sub = df.copy()
    sub["predicted_positions"] = preds_str
    return sub[["id", "predicted_positions"]]

# генерация submission.csv
submission = make_submission(df, model, thr=0.55).sort_values("id")
submission.to_csv("submission.csv", index=False)
print("Готово: submission.csv")

# для быстрой проверки первых строк
print(submission.head(10))

Готово: submission.csv
   id predicted_positions
0   0         [5, 10, 12]
1   1          [3, 6, 10]
2   2         [4, 12, 20]
3   3         [5, 10, 18]
4   4             [5, 10]
5   5             [6, 14]
6   6         [5, 14, 19]
7   7         [3, 12, 15]
8   8         [6, 13, 16]
9   9             [5, 12]
